Convert gt.txt to YOLO Labels for Train and Val

In [1]:
import os
import cv2

# Paths
base_dir = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013"
gt_file = os.path.join(base_dir, "gt.txt")

# Helper: which images are in train/val
train_images_dir = os.path.join(base_dir, "train", "images")
val_images_dir = os.path.join(base_dir, "val", "images")
train_labels_dir = os.path.join(base_dir, "train", "labels")
val_labels_dir = os.path.join(base_dir, "val", "labels")
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

train_images = set(os.listdir(train_images_dir))
val_images = set(os.listdir(val_images_dir))

with open(gt_file, 'r') as f:
    for line in f:
        parts = line.strip().split(';')
        if len(parts) < 6: continue
        img_name, left, top, right, bottom, class_id = parts[0], int(parts[1]), int(parts[2]), int(parts[3]), int(parts[4]), int(parts[5])
        # Decide if this image is train or val
        if img_name in train_images:
            labels_dir = train_labels_dir
            images_dir = train_images_dir
        elif img_name in val_images:
            labels_dir = val_labels_dir
            images_dir = val_images_dir
        else:
            continue  # Not in either split
        img_path = os.path.join(images_dir, img_name)
        if not os.path.exists(img_path): continue
        img = cv2.imread(img_path)
        if img is None: continue
        height, width = img.shape[:2]
        x_center = ((left + right) / 2) / width
        y_center = ((top + bottom) / 2) / height
        box_width = (right - left) / width
        box_height = (bottom - top) / height
        label_path = os.path.join(labels_dir, os.path.splitext(img_name)[0] + '.txt')
        with open(label_path, 'a') as label_file:
            label_file.write(f"{class_id} {x_center} {y_center} {box_width} {box_height}\n")


Create the Required Folder Structure

In [3]:
import os

# Base directory path
base_dir = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013"

# Create folder structure
folders = [
    "train/images",
    "train/labels", 
    "val/images",
    "val/labels"
]

for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    os.makedirs(folder_path, exist_ok=True)
    print(f"Created: {folder_path}")


Created: C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013\train/images
Created: C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013\train/labels
Created: C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013\val/images
Created: C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013\val/labels


Move Your Images to the Correct Folders

In [4]:
import shutil
import os

# Source directory (where your original images are)
source_images = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013"  # Update this path

# Destination directories
train_images_dir = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/train/images"
val_images_dir = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/val/images"

# Move training images (00000.ppm to 00479.ppm)
for i in range(480):
    img_name = f"{i:05d}.ppm"
    src_path = os.path.join(source_images, img_name)
    dst_path = os.path.join(train_images_dir, img_name)
    
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)
        print(f"Moved {img_name} to train")

# Move validation images (00480.ppm to 00599.ppm)
for i in range(480, 600):
    img_name = f"{i:05d}.ppm"
    src_path = os.path.join(source_images, img_name)
    dst_path = os.path.join(val_images_dir, img_name)
    
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)
        print(f"Moved {img_name} to val")


Convert Annotations to YOLO Format

In [5]:
import os
import cv2

# Paths
base_dir = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013"
gt_file = os.path.join(base_dir, "gt.txt")

# Destination directories
train_images_dir = os.path.join(base_dir, "train", "images")
val_images_dir = os.path.join(base_dir, "val", "images")
train_labels_dir = os.path.join(base_dir, "train", "labels")
val_labels_dir = os.path.join(base_dir, "val", "labels")

# Get list of images in each folder
train_images = set(os.listdir(train_images_dir)) if os.path.exists(train_images_dir) else set()
val_images = set(os.listdir(val_images_dir)) if os.path.exists(val_images_dir) else set()

print(f"Train images: {len(train_images)}, Val images: {len(val_images)}")

# Process annotations
with open(gt_file, 'r') as f:
    for line in f:
        parts = line.strip().split(';')
        if len(parts) < 6: continue
        
        img_name, left, top, right, bottom, class_id = parts[0], int(parts[1]), int(parts[2]), int(parts[3]), int(parts[4]), int(parts[5])
        
        # Determine if image is in train or val
        if img_name in train_images:
            labels_dir = train_labels_dir
            images_dir = train_images_dir
        elif img_name in val_images:
            labels_dir = val_labels_dir
            images_dir = val_images_dir
        else:
            continue  # Skip if image not found
            
        # Read image to get dimensions
        img_path = os.path.join(images_dir, img_name)
        if not os.path.exists(img_path): continue
        
        img = cv2.imread(img_path)
        if img is None: continue
        
        height, width = img.shape[:2]
        
        # Convert to YOLO format
        x_center = ((left + right) / 2) / width
        y_center = ((top + bottom) / 2) / height
        box_width = (right - left) / width
        box_height = (bottom - top) / height
        
        # Write label file
        label_path = os.path.join(labels_dir, os.path.splitext(img_name)[0] + '.txt')
        with open(label_path, 'a') as label_file:
            label_file.write(f"{class_id} {x_center} {y_center} {box_width} {box_height}\n")

print("Annotation conversion complete!")


Train images: 480, Val images: 120
Annotation conversion complete!


Verify Your Folder Structure

In [6]:
import os

base_dir = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013"

# Check folder contents
for folder in ["train/images", "train/labels", "val/images", "val/labels"]:
    folder_path = os.path.join(base_dir, folder)
    if os.path.exists(folder_path):
        files = os.listdir(folder_path)
        print(f"{folder}: {len(files)} files")
    else:
        print(f"{folder}: DOES NOT EXIST")


train/images: 480 files
train/labels: 443 files
val/images: 120 files
val/labels: 63 files


Locate Your Actual Image Files

In [8]:
import os
import glob

# Search for image files in broader locations
search_paths = [
    "C:/Users/Venu Gopal/Downloads/**/*",
    "C:/Users/Venu Gopal/Desktop/**/*",
    "C:/Users/Venu Gopal/Documents/**/*"
]

print("Searching for traffic sign images...")
for search_path in search_paths:
    # Search for common image formats
    for ext in ['*.ppm', '*.jpg', '*.png', '*.jpeg']:
        files = glob.glob(search_path + ext, recursive=True)
        if files:
            print(f"Found {len(files)} {ext} files in {search_path}")
            print(f"Sample files: {files[:5]}")
            break


Searching for traffic sign images...
Found 1753 *.ppm files in C:/Users/Venu Gopal/Downloads/**/*
Sample files: ['C:/Users/Venu Gopal/Downloads\\TestIJYOLO2013\\TestIJYOLO2013Download\\images\\00000.ppm', 'C:/Users/Venu Gopal/Downloads\\TestIJYOLO2013\\TestIJYOLO2013Download\\images\\00001.ppm', 'C:/Users/Venu Gopal/Downloads\\TestIJYOLO2013\\TestIJYOLO2013Download\\images\\00002.ppm', 'C:/Users/Venu Gopal/Downloads\\TestIJYOLO2013\\TestIJYOLO2013Download\\images\\00003.ppm', 'C:/Users/Venu Gopal/Downloads\\TestIJYOLO2013\\TestIJYOLO2013Download\\images\\00004.ppm']
Found 9508 *.jpg files in C:/Users/Venu Gopal/Desktop/**/*
Sample files: ['C:/Users/Venu Gopal/Desktop\\Files\\SEM 3\\MTH301\\Exercises\\0_p_1 case.jpg', 'C:/Users/Venu Gopal/Desktop\\Files\\SEM 3\\MTH301\\Exercises\\Distance formula (Stereographic projection) - 1.jpg', 'C:/Users/Venu Gopal/Desktop\\Files\\SEM 3\\MTH301\\Exercises\\Distance formula (Stereographic projection) - 2.jpg', 'C:/Users/Venu Gopal/Desktop\\Files\\SE

Check Your Downloaded Data

In [9]:
# Check what's actually in your TrainIJYOLO2013 folder
base_path = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013"
print("Contents of TrainIJYOLO2013 folder:")
for root, dirs, files in os.walk(base_path):
    print(f"Directory: {root}")
    print(f"Subdirectories: {dirs}")
    print(f"Files: {files[:10]}")  # Show first 10 files
    print("-" * 50)


Contents of TrainIJYOLO2013 folder:
Directory: C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013
Subdirectories: ['TrainIJYOLO2013']
Files: []
--------------------------------------------------
Directory: C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013\TrainIJYOLO2013
Subdirectories: ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 'train', 'val']
Files: ['ex.txt', 'gt.txt', 'ReadMeTrain.txt', 'traffic_signs.yaml']
--------------------------------------------------
Directory: C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013\TrainIJYOLO2013\00
Subdirectories: []
Files: ['00000.ppm', '00001.ppm', '00002.ppm', '00003.ppm']
--------------------------------------------------
Directory: C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013\TrainIJYOLO2013\01
Subdirectories: []
Files: ['00000.ppm', '000

Move Images to Correct Folders 

In [10]:
import shutil
import os

# Update this path to where your images actually are
actual_images_location = "path/to/your/actual/images"  # UPDATE THIS!

base_dir = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013"
train_images_dir = os.path.join(base_dir, "train/images")
val_images_dir = os.path.join(base_dir, "val/images")

# Move training images (00000.ppm to 00479.ppm)
for i in range(480):
    img_name = f"{i:05d}.ppm"
    src = os.path.join(actual_images_location, img_name)
    dst = os.path.join(train_images_dir, img_name)
    if os.path.exists(src):
        shutil.copy2(src, dst)  # Use copy2 instead of move to keep originals
        print(f"Copied {img_name} to train")

# Move validation images (00480.ppm to 00599.ppm)
for i in range(480, 600):
    img_name = f"{i:05d}.ppm"
    src = os.path.join(actual_images_location, img_name)
    dst = os.path.join(val_images_dir, img_name)
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print(f"Copied {img_name} to val")


Convert Images

In [12]:
import os
from PIL import Image

def convert_ppm_to_jpg(folder):
    for filename in os.listdir(folder):
        if filename.endswith('.ppm'):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path)
            new_filename = filename.replace('.ppm', '.jpg')
            img.save(os.path.join(folder, new_filename))
            print(f"Converted {filename} to {new_filename}")

# Convert train and val images
convert_ppm_to_jpg("C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/train/images")
convert_ppm_to_jpg("C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/val/images")


Converted 00000.ppm to 00000.jpg
Converted 00001.ppm to 00001.jpg
Converted 00002.ppm to 00002.jpg
Converted 00003.ppm to 00003.jpg
Converted 00004.ppm to 00004.jpg
Converted 00005.ppm to 00005.jpg
Converted 00006.ppm to 00006.jpg
Converted 00007.ppm to 00007.jpg
Converted 00008.ppm to 00008.jpg
Converted 00009.ppm to 00009.jpg
Converted 00010.ppm to 00010.jpg
Converted 00011.ppm to 00011.jpg
Converted 00012.ppm to 00012.jpg
Converted 00013.ppm to 00013.jpg
Converted 00014.ppm to 00014.jpg
Converted 00015.ppm to 00015.jpg
Converted 00016.ppm to 00016.jpg
Converted 00017.ppm to 00017.jpg
Converted 00018.ppm to 00018.jpg
Converted 00019.ppm to 00019.jpg
Converted 00020.ppm to 00020.jpg
Converted 00021.ppm to 00021.jpg
Converted 00022.ppm to 00022.jpg
Converted 00023.ppm to 00023.jpg
Converted 00024.ppm to 00024.jpg
Converted 00025.ppm to 00025.jpg
Converted 00026.ppm to 00026.jpg
Converted 00027.ppm to 00027.jpg
Converted 00028.ppm to 00028.jpg
Converted 00029.ppm to 00029.jpg
Converted 

Remove Old .ppm Images

In [13]:
import os

def remove_ppm(folder):
    for filename in os.listdir(folder):
        if filename.endswith('.ppm'):
            os.remove(os.path.join(folder, filename))
            print(f"Removed {filename}")

remove_ppm("C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/train/images")
remove_ppm("C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/val/images")


Removed 00000.ppm
Removed 00001.ppm
Removed 00002.ppm
Removed 00003.ppm
Removed 00004.ppm
Removed 00005.ppm
Removed 00006.ppm
Removed 00007.ppm
Removed 00008.ppm
Removed 00009.ppm
Removed 00010.ppm
Removed 00011.ppm
Removed 00012.ppm
Removed 00013.ppm
Removed 00014.ppm
Removed 00015.ppm
Removed 00016.ppm
Removed 00017.ppm
Removed 00018.ppm
Removed 00019.ppm
Removed 00020.ppm
Removed 00021.ppm
Removed 00022.ppm
Removed 00023.ppm
Removed 00024.ppm
Removed 00025.ppm
Removed 00026.ppm
Removed 00027.ppm
Removed 00028.ppm
Removed 00029.ppm
Removed 00030.ppm
Removed 00031.ppm
Removed 00032.ppm
Removed 00033.ppm
Removed 00034.ppm
Removed 00035.ppm
Removed 00036.ppm
Removed 00037.ppm
Removed 00038.ppm
Removed 00039.ppm
Removed 00040.ppm
Removed 00041.ppm
Removed 00042.ppm
Removed 00043.ppm
Removed 00044.ppm
Removed 00045.ppm
Removed 00046.ppm
Removed 00047.ppm
Removed 00048.ppm
Removed 00049.ppm
Removed 00050.ppm
Removed 00051.ppm
Removed 00052.ppm
Removed 00053.ppm
Removed 00054.ppm
Removed 00

Install YOLOv8 (Ultralytics) and Train

In [4]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
results = model.train(
    data="C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/traffic_signs.yaml",
    epochs=100,
    imgsz=640
)


100%|██████████| 6.25M/6.25M [00:01<00:00, 5.20MB/s]


Ultralytics 8.3.161  Python-3.11.9 torch-2.7.1+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/traffic_signs.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pat

train: Scanning C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\train\labels.cache... 443 images, 37 backgrounds, 0 corrupt: 100%|██████████| 480/480 [00:00<?, ?it/s]

train: C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\train\images\00000.jpg: 1 duplicate labels removed
train: C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\train\images\00001.jpg: 3 duplicate labels removed
train: C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\train\images\00002.jpg: 1 duplicate labels removed
train: C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\train\images\00003.jpg: 3 duplicate labels removed
train: C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\train\images\00004.jpg: 2 duplicate labels removed
train: C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\train\images\00005.jpg: 1 duplicate labels removed
train: C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\train\images\00006.jpg: 1 duplicate labels removed
train: C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\train\images\00007.jpg: 1 duplicate labels removed
train: C:\Users\Venu Gopal\Downloads\Tra


c:\Users\Venu Gopal\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\Venu Gopal\Downloads\TrainIJYOLO2013\TrainIJYOLO2013\val\labels.cache... 63 images, 57 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]

...

     33/100         0G     0.8028      2.017     0.8135         48        640: 100%|██████████| 30/30 [04:32<00:00,  9.10s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:13<00:00,  3.25s/it]

                   all        120         98      0.465      0.414      0.327      0.247



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     34/100         0G     0.9038       2.11     0.8214         38        640:  47%|████▋     | 14/30 [02:40<03:03, 11.45s/it]


KeyboardInterrupt: 

Evaluate and Predict

In [ ]:
# Load trained model
model = YOLO("runs/detect/train6/weights/best.pt")

# Predict on a sample image from val set
img_path = "C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/val/images/00501.ppm" #need to convert to jpg, my bad
results = model.predict(img_path, imgsz=640)
results[0].show()  # Show image with boxes


FileNotFoundError: C:/Users/Venu Gopal/Downloads/TrainIJYOLO2013/TrainIJYOLO2013/val/images/00501.ppm does not exist